In [ ]:
import kaggle
import json
import os
import pandas as pd

from gensim.parsing.preprocessing import (strip_tags, strip_punctuation, strip_numeric, 
                                          stem_text, strip_multiple_whitespaces, 
                                          strip_non_alphanum, remove_stopwords, strip_short)

from gensim import corpora, models, similarities

import spacy

import pyLDAvis

import pyLDAvis.gensim as gensimvis

In [69]:
nlp = spacy.load('en_core_web_lg')

In [2]:
# Connecting to kaggle api to download dataset
dataset='allen-institute-for-ai/CORD-19-research-challenge'
kaggle.api.dataset_download_files(dataset=dataset, unzip=True, path='data')

In [4]:
path = 'data/biorxiv_medrxiv/biorxiv_medrxiv/pdf_json/'
files = os.listdir(path=path)
# print(len(files))

In [68]:
df = pd.DataFrame()

for file in files[:]:
    file_path = path + file
    
    file_data_min = {}
    
    with open(file_path) as f:
        file_data = f.read()
        file_data = json.loads(file_data)
    
    
    file_data_min['paper_id'] = file_data['paper_id']
    
    
#     abstract = file_data['abstract']
#     abstract_min = ''
#     for i in abstract:
#         abstract_min += i['text'] + ' '
#     file_data_min['abstract'] = abstract_min
    
    
    body_text = file_data['body_text']
    body_text_min = ''
    for i in body_text:
        body_text_min += i['text'] + ' '
#     file_data_min['body_text'] = body_text_min
    
    
    doc = nlp(body_text_min)
    

    file_data_min['doc'] = doc

    
    for i in doc.ents[:]:

        entity_label = i.label_

        if entity_label in file_data_min:
            file_data_min[entity_label] += 1
        else:
            file_data_min[entity_label] = 1
    
    
    df = df.append(file_data_min, ignore_index=True)


print(len(df))


df.head(3)

2278


,CARDINAL,DATE,EVENT,FAC,GPE,MONEY,NORP,ORDINAL,ORG,PERCENT,PERSON,PRODUCT,QUANTITY,TIME,doc,paper_id,LAW,WORK_OF_ART,LANGUAGE,LOC
0,119.0,32.0,1.0,1.0,8.0,12.0,1.0,5.0,52.0,28.0,23.0,4.0,27.0,6.0,"(Colorectal, cancer, is, globally, the, third,...",f6b29be971089bfe0916c64ab9fbddcec38a7436,NaN,NaN,NaN,NaN
1,101.0,51.0,NaN,1.0,65.0,NaN,NaN,1.0,6.0,3.0,14.0,11.0,NaN,2.0,"(The, outbreak, of, infectious, diseases, has,...",005d189d5bd7ac01aee65e934fd3d5186a3f7b27,3.0,2.0,NaN,NaN
2,143.0,12.0,1.0,NaN,2.0,NaN,1.0,NaN,116.0,5.0,12.0,37.0,8.0,1.0,"(In, the, absence, of, a, curative, treatment,...",f905f78b32f63c6d14a79984dfb33f1b358b8ab4,NaN,6.0,1.0,NaN


In [76]:
type(df['doc'][0])

spacy.tokens.doc.Doc

In [106]:
matcher = spacy.matcher.PhraseMatcher(nlp.vocab)

In [99]:
len(doc)

4808

In [122]:
phrase_list = ['cancer', 'covid']

phrase_list = [nlp(text) for text in phrase_list]

matcher.add('search_items1', None, *phrase_list)

doc = df['doc'][0]

found_matches = matcher(doc)
print(found_matches)

context_words = 10

for matcher_id, start, end in found_matches:
    string_text = nlp.vocab.strings[matcher_id]
    
    if start > context_words:
        startX = start - context_words
    else:
        startx = start
        
    if len(doc) - end < context_words:
        endX = len(doc)
    else:
        endX = end + context_words
    
    span = doc[startX : endX]
    
    print(f'{startX} {endX} {span.text}')
    
matcher.remove('search_items1')

[(1412477739064755821, 1, 2), (1412477739064755821, 189, 190), (1412477739064755821, 560, 561), (1412477739064755821, 607, 608), (1412477739064755821, 3342, 3343), (1412477739064755821, 3676, 3677), (1412477739064755821, 3883, 3884), (1412477739064755821, 4568, 4569)]
4566 12 
179 200 to comply with the 2015 NG12 NICE guidance for suspected cancer (6) , which recommended the use of faecal
550 571 7) . While introduction of FIT into primary care cancer pathways is limited at present, given the large backlog
597 618 has passed to risk stratify all patient referred with possible cancer symptoms into groups for urgent and less urgent endoscopy.
3332 3353 Neither of these patients have had a diagnosis of colorectal cancer, more than 1 year after the specimens were first
3666 3687 ) of the utility of two sample strategies for colorectal cancer in a symptomatic population it was noted that there was
3873 3894 that the majority of FIT results from patients with colorectal cancer had very high 

In [114]:
type(found_matches[0][1])

int

In [119]:
doc0 = nlp(df['body_text'][0])

doc0_tokens = [token for token in doc0]

KeyError: 'body_text'

In [50]:
doc0.ents

entity_counter = {}

for ent in doc0.ents[:]:
    
    entity_label = ent.label_
    
    if entity_label in entity_counter:
        entity_counter[entity_label] += 1
    else:
        entity_counter[entity_label] = 1
        
entity_counter

{'ORDINAL': 5,
 'CARDINAL': 119,
 'GPE': 8,
 'ORG': 52,
 'DATE': 32,
 'PERSON': 23,
 'FAC': 1,
 'MONEY': 12,
 'TIME': 6,
 'QUANTITY': 27,
 'PRODUCT': 4,
 'PERCENT': 28,
 'NORP': 1,
 'EVENT': 1}

In [35]:
counter = {}

In [38]:
counter['item1'] = 1
counter

{'item1': 1}

In [45]:
'item2' in counter

False

In [125]:
spacy.displacy.render(docs = df['doc'][0], style='ent', page=False, minify=False, )

In [137]:
%%time

docs = []

for doc in df['doc'][:3]:
    tokens = [token.text for token in doc]
    docs.append(tokens)

len(docs)

dictionary = corpora.Dictionary(docs)

corpus = [dictionary.doc2bow(text) for text in docs]

CPU times: user 27.1 ms, sys: 2.51 ms, total: 29.6 ms
Wall time: 30.5 ms


In [138]:
[ [ (dictionary[id], freq) for id , freq in cp ] for cp in corpus[:1] ]

[[(' ', 1),
  ('"', 6),
  ('%', 35),
  ("'s", 4),
  ('(', 95),
  (')', 94),
  ('+', 2),
  (',', 133),
  ('-', 33),
  ('-%cv', 1),
  ('-524', 1),
  ('-As', 1),
  ('-LOQ', 1),
  ('-The', 2),
  ('-y', 1),
  ('.', 167),
  ('/', 61),
  ('0', 1),
  ('0.10', 1),
  ('0.16', 1),
  ('0.20', 1),
  ('0.21', 1),
  ('0.35', 1),
  ('0.47', 1),
  ('0.5', 4),
  ('0.5µg', 1),
  ('0.84', 1),
  ('1', 7),
  ('1.0', 2),
  ('1.1', 7),
  ('1.5', 2),
  ('1.645', 1),
  ('1.645SD', 1),
  ('10', 25),
  ('10%CV', 2),
  ('10.2', 2),
  ('100', 2),
  ('11', 2),
  ('11.5', 1),
  ('12', 3),
  ('12.4', 1),
  ('12.5', 1),
  ('120', 1),
  ('13', 2),
  ('13.5', 2),
  ('132', 1),
  ('14', 1),
  ('14.7', 1),
  ('15', 6),
  ('15.0', 1),
  ('15.20066191', 1),
  ('150', 1),
  ('16', 1),
  ('17', 1),
  ('18', 1),
  ('188', 1),
  ('19', 1),
  ('197', 1),
  ('2', 10),
  ('20', 1),
  ('20.5', 1),
  ('2015', 1),
  ('2016', 2),
  ('2017', 1),
  ('21', 5),
  ('222', 1),
  ('225', 1),
  ('25', 2),
  ('26', 1),
  ('27', 1),
  ('27.8', 2

In [129]:
# notes --> remove stopwords and punctuation before creating dictionary

In [139]:
tfidf = models.TfidfModel(corpus)

In [140]:
transformed_tfidf = tfidf[corpus]

In [141]:
%time lda = models.LdaMulticore(transformed_tfidf, num_topics=10, id2word=dictionary)

CPU times: user 167 ms, sys: 121 ms, total: 287 ms
Wall time: 200 ms


In [142]:
lda.show_topics()

[(0,
  '0.000*"IN" + 0.000*"CTD" + 0.000*"daily" + 0.000*"g" + 0.000*"CCD" + 0.000*"rate" + 0.000*"binding" + 0.000*"site-2" + 0.000*"FIT" + 0.000*"F"'),
 (1,
  '0.003*"daily" + 0.002*"rate" + 0.001*"cumulative" + 0.001*"\uf0b7" + 0.001*"average" + 0.001*"F" + 0.001*"temperature" + 0.001*"epidemic" + 0.001*"significantly" + 0.001*"case"'),
 (2,
  '0.000*"IN" + 0.000*"CTD" + 0.000*"daily" + 0.000*"g" + 0.000*"CCD" + 0.000*"cumulative" + 0.000*"rate" + 0.000*"binding" + 0.000*"peptide" + 0.000*"µg"'),
 (3,
  '0.000*"IN" + 0.000*"CTD" + 0.000*"daily" + 0.000*"g" + 0.000*"peptide" + 0.000*"rate" + 0.000*"µg" + 0.000*"cumulative" + 0.000*"site-2" + 0.000*"binding"'),
 (4,
  '0.000*"IN" + 0.000*"CTD" + 0.000*"daily" + 0.000*"g" + 0.000*"µg" + 0.000*"cumulative" + 0.000*"binding" + 0.000*"average" + 0.000*"CCD" + 0.000*"rate"'),
 (5,
  '0.000*"IN" + 0.000*"daily" + 0.000*"CTD" + 0.000*"cumulative" + 0.000*"g" + 0.000*"µg" + 0.000*"F" + 0.000*"rate" + 0.000*"FIT" + 0.000*"site-2"'),
 (6,
  '0.

In [143]:
pyLDAvis.enable_notebook()

In [ ]:
vis_data = gensimvis.prepare(lda, corpus, dictionary)

pyLDAvis.show(vis_data)


Note: if you're in the IPython notebook, pyLDAvis.show() is not the best command
      to use. Consider using pyLDAvis.display(), or pyLDAvis.enable_notebook().
      See more information at http://pyLDAvis.github.io/quickstart.html .

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8889/    [Ctrl-C to exit]


/Users/moseswamambo/venv/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
127.0.0.1 - - [03/May/2020 14:21:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2020 14:21:08] "GET /LDAvis.css HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2020 14:21:08] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2020 14:21:08] "GET /LDAvis.js HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2020 14:21:08] code 404, message Not Found
127.0.0.1 - - [03/May/2020 14:21:08] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [03/May/2020 14:21:08] code 404, message Not Found
127.0.0.1 - - [03/May/2020 14:21:08] "GET /apple-touch-icon-precomposed.png HTTP/1.1" 404 -
127.0.0.1 - - [03/May/2020 14:21:08] code 404,

In [9]:
CUSTOM_FILTERS = [lambda x: x.lower(),
                             strip_multiple_whitespaces,
                             strip_numeric,
                             remove_stopwords,
                             strip_short,
                             stem_text,
                             strip_tags,
                             strip_punctuation
                 ]

In [ ]:
doc = preprocess_string(df['body_text'][0], CUSTOM_FILTERS)

In [ ]:
dictionary = corpora.Dictionary(['test', 'testing'])